In [1]:
import shutil
import os

import netCDF4 as nc
import pandas as pd
import xarray as xr
import numpy as np

#### Stages to make Latin Hypercube

 - Make an ensemble of files
 - Copy one of the SP calibration outputs into each of these file
 - Make a list of variables to modify
 - Create actual hypercube
 - Define the ranges of the parameters using the csv file
 - Apply to parameter list 

#### Control paths

In [5]:
# Averaged SP calibrated values, from paraemter_averaging and paraemeter_merge scripts. 
averaged_SP_params_files = '../fates_params_vertsc_dayl_SPcal.nc' 

# Where to find the list of parameters and their modifications
parameter_list_file = '../csvfiles_for_parameter_modification/NOCOMP_LHC_parameters.csv'
pft_parameter_list = '../csvfiles_for_parameter_modification/pft_specific_parameter_inputs/PFT_params.csv'

# Adjusted default x calibration file to make a reasonable baseline. 
modified_baseline_param_files='../intermediate_pfiles/modified_average_file2.nc'

# What is the path to the new parameter file ensemble 
pdirroot='../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_'

# SP parameter files path 
sp_posterior_filepath='../lhc_param_files/'

#### Read the main parameter list CSV file into a pandas DataFrame

In [7]:
df = pd.read_csv(parameter_list_file)
fates_parameter_list = df['fates_parameter'].tolist()
minp = df['min'].tolist() 
maxp = df['max'].tolist() 
dimp = df['dim'].tolist()

#### Make a dictionary to index the ranges searchable by parameter name

In [9]:
pftname_vs_csvfileindex = {
    "fates_rad_stem_rhovis": "rhosvis",
    "fates_rad_stem_rhonir": "rhosnir",
    "fates_rad_leaf_rhonir": "rholnir",
    "fates_rad_leaf_rhovis": "rholvis",
    "fates_rad_leaf_taunir": "taulnir",
    "fates_rad_leaf_tauvis": "taulvis",
    "fates_leaf_stomatal_slope_medlyn": "medlynslope",
    "fates_leaf_slatop": "sla",
    "fates_rad_leaf_xl": "xl"   
}

In [10]:
# index the order of the parameters in the csv input file. 
my_dict = {}
print(len(fates_parameter_list))
for i in range(len(fates_parameter_list)):
    my_dict[fates_parameter_list[i]] = i


121


#### Read the PFT parameter list CSV file into a pandas DataFrame

In [12]:
dfp = pd.read_csv(pft_parameter_list)
pftindex = dfp['pft_index'].tolist()
pftparam = dfp['param'].tolist() 
pftmin = dfp['min'].tolist()
pftmax = dfp['max'].tolist()



#### map from the variables in the pftvariable csv file into the fates pft space

In [14]:
string_mapping = {
    "fates_rad_stem_rhovis": "rhosvis",
    "fates_rad_stem_rhonir": "rhosnir",
    "fates_rad_leaf_rhonir": "rholnir",
    "fates_rad_leaf_rhovis": "rholvis",
    "fates_rad_leaf_taunir": "taulnir",
    "fates_rad_leaf_tauvis": "taulvis",
    "fates_leaf_stomatal_slope_medlyn": "medlynslope",
    "fates_leaf_slatop": "sla",
    "fates_rad_leaf_xl": "xl"   
}

In [15]:
# index the order of the pfts in the input file. 
my_dict_pft = {}
print(len(pftparam))
for j in range(1,len(pftparam)):
    my_dict_pft[pftparam[j]] = j
print(my_dict_pft)

84
{'taulnir': 11, 'taulvis': 23, 'rholnir': 35, 'rholvis': 47, 'xl': 59, 'medlynslope': 71, 'sla': 83}


#### Retreive the full list of the parameters from the csv file 

In [17]:
reference_vars = nc.Dataset(modified_baseline_param_files, 'r+') 
fatespfts=reference_vars.dimensions['fates_pft']
print(fatespfts.size)


12


#### Find and read the SP parameter files. 

In [19]:
file_list = os.listdir(sp_posterior_filepath)
sp_posterior=[]
for file in file_list:
    file_path = os.path.join(sp_posterior_filepath, file)
    dataset = xr.open_dataset(file_path)
    sp_posterior.append(dataset)



#### Variables to modify chosen from OAAT analysis 

In [21]:
allvars=[]
laivars= ['fates_leaf_slatop', 'fates_allom_d2bl1', 'fates_allom_d2ca_coefficient_max', 'fates_leaf_slamax', 'fates_stoich_nitr', 'fates_recruit_seed_supplement', 'fates_q10_mr', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_phen_cold_size_threshold', 'fates_turnover_leaf', 'fates_turnover_fnrt', 'fates_alloc_storage_cushion']
nppvars= ['fates_stoich_nitr', 'fates_leaf_slatop', 'fates_leafn_vert_scaler_coeff2', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_q10_mr', 'fates_grperc', 'fates_phen_gddthresh_b', 'fates_phen_gddthresh_b', 'fates_leaf_slamax', 'fates_turnover_leaf', 'fates_maintresp_leaf_vert_scaler_coeff1']
vegcvars=['fates_stoich_nitr', 'fates_leaf_slatop', 'fates_alloc_storage_cushion', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_q10_mr', 'fates_allom_d2bl2', 'fates_turnover_fnrt',  'fates_leafn_vert_scaler_coeff2', 'fates_grperc', 'fates_recruit_seed_supplement', 'fates_recruit_seed_alloc']
extravars = ['fates_phen_coldtemp','fates_phen_drought_threshold','fates_allom_fnrt_prof_b',
                   'fates_mort_freezetol','fates_phen_mindayson',
            'fates_mort_hf_sm_threshold','fates_mort_scalar_coldstress',
            'fates_mort_scalar_cstarvation','fates_mort_scalar_hydrfailure']

nocomp_lhc_params = list(set(laivars + nppvars + vegcvars+extravars))
print(nocomp_lhc_params)
print(len(nocomp_lhc_params))

['fates_leaf_slatop', 'fates_allom_fnrt_prof_b', 'fates_turnover_fnrt', 'fates_allom_d2bl2', 'fates_phen_drought_threshold', 'fates_phen_mindayson', 'fates_alloc_storage_cushion', 'fates_phen_gddthresh_b', 'fates_mort_scalar_hydrfailure', 'fates_turnover_leaf', 'fates_grperc', 'fates_stoich_nitr', 'fates_leafn_vert_scaler_coeff2', 'fates_q10_mr', 'fates_allom_d2bl1', 'fates_recruit_seed_supplement', 'fates_allom_d2ca_coefficient_max', 'fates_mort_freezetol', 'fates_maintresp_leaf_vert_scaler_coeff1', 'fates_mort_hf_sm_threshold', 'fates_recruit_seed_alloc', 'fates_maintresp_leaf_vert_scaler_coeff2', 'fates_phen_cold_size_threshold', 'fates_mort_scalar_cstarvation', 'fates_phen_coldtemp', 'fates_leaf_slamax', 'fates_mort_scalar_coldstress']


#### Make a latin hypercube

In [23]:
nlhc_samples = 100
num_variables = len(nocomp_lhc_params)

# Generate a Latin Hypercube
hypercube = np.zeros((nlhc_samples, num_variables))

for i in range(num_variables):
    hypercube[:, i] = np.random.permutation(np.linspace(0, 1, nlhc_samples))


#### Modify param files with nocomp modifications in LHC

In [25]:
print('nlhc',nlhc_samples)
for i in range(1,nlhc_samples):
    # Make a new filw for each ensemble member
    counter = 0
    newfile = pdirroot + str(i) + '.nc' 
    print(newfile)
    shutil.copy(modified_baseline_param_files, newfile)
    nc_file = nc.Dataset(newfile, 'r+')
    min_range_pft=range(1,12)
    max_range_pft=range(1,12)    
    #print(nc_file.variables.keys())
    # Perform operations on each target lhc parameter  
    for index, parameter in enumerate(nocomp_lhc_params):
        if parameter in reference_vars.variables:
            variable_to_modify = nc_file.variables[parameter]
            csvindex=my_dict[parameter]
            print('#index',i,index, parameter,minp[csvindex],maxp[csvindex])  # Replace with your operations
            # Find the range for each parameter depending on its type. 
            if(variable_to_modify.ndim==1):
                if((variable_to_modify.size)>2):
                    variable_data = variable_to_modify[:]
                else:
                    print('shortvar')
                    variable_data = variable_to_modify[:]           
            elif variable_to_modify.ndim==0:
                variable_data = variable_to_modify[:]
            else:
                variable_data = variable_to_modify[:,:]               

            if minp[csvindex] == '50percent' or minp[csvindex] == '100percent':
                if minp[csvindex] == '50percent': 
                    perc_ch=0.5
                if minp[csvindex] == '100percent': 
                    perc_ch=1.0
                if not variable_to_modify.dtype == int:
                    min_range = (variable_data) * float(perc_ch)
                    max_range = variable_data * (1+  (perc_ch))
                else: # integer 
                    min_range = (xc * perc_ch).astype(int)
                    max_range = (variable_data * (1+ perc_ch)).astype(int)
            elif not minp[csvindex] ==  'pft':
                min_range  = variable_data*0 +float(minp[csvindex])
                max_range = variable_data*0 +float(maxp[csvindex])
            elif  minp[csvindex] ==  'pft':
                shortvar = string_mapping[parameter]
                table = dfp[dfp['param'] == shortvar]
                pftmins=table['min'].tolist()
                pftmaxs=table['min'].tolist()
                print('pftmins',pftmins)
                variable_to_modify[:] *=  0
                for p in range(1, 12 ):               
                    min_range_pft[p]  = variable_data*0 +float(float(pftmins[p-1]))
                    max_range_pft[p]  = variable_data*0 +float(float(pftmaxs[p-1]))
#-------------------------------------------------------------
# Change variable according to LHC value
#-------------------------------------------------------------
            print('hypercube',hypercube[i, index])
            if not minp[csvindex] ==  'pft': 
            # Modify variables according to the LHC ranges

                rangev = max_range-min_range
                hrangev = rangev* hypercube[i, index]
                newvalue = min_range + hrangev
               # print(variable_to_modify.values,newvalue)
                variable_to_modify = newvalue
                
            else: 
                for p in range(1, 12 ): 
                    range = max_range_pft[p]-min_range_pft[p]
                    hrange = range* hypercube[i, index]
                    newvalue = min_range + hrange
                    variable_to_modify[p] = newvalue 
            nc_file.variables[parameter][:] = variable_to_modify

        else: #doenst exist 
            print('target variable is not in csv file', parameter)
    nc_file.close()

nlhc 100
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_1.nc
#index 1 0 fates_leaf_slatop 50percent 50percent
hypercube 0.020202020202020204
#index 1 1 fates_allom_fnrt_prof_b 50percent 50percent
hypercube 0.8787878787878789
#index 1 2 fates_turnover_fnrt 50percent 50percent
hypercube 0.3535353535353536
#index 1 3 fates_allom_d2bl2 1.0 2.0
hypercube 0.3434343434343435
#index 1 4 fates_phen_drought_threshold 50percent 50percent
hypercube 0.5858585858585859
#index 1 5 fates_phen_mindayson 50percent 50percent
hypercube 0.020202020202020204
#index 1 6 fates_alloc_storage_cushion 50percent 50percent
hypercube 0.8080808080808082
#index 1 7 fates_phen_gddthresh_b 50percent 50percent
hypercube 1.0
#index 1 8 fates_mort_scalar_hydrfailure 0.5 6
hypercube 0.5656565656565657
#index 1 9 fates_turnover_leaf 50percent 50percent
hypercube 0.020202020202020204
#index 1 10 fates_grperc 0.1 0.3
hypercube 0.5353535353535354
#index 1 11 fates_stoich_nitr 50percent 50percent
hypercube 0.9696969696969697
#index 1 1

#### check diffs between nocomp lhc members 1st

In [27]:
newfile = pdirroot + str(1) + '.nc' 
nc_file_1 = nc.Dataset(newfile, 'r+')

for i in range(1,nlhc_samples):
    newfile = pdirroot + str(i) + '.nc' 
    nc_file_i = nc.Dataset(newfile, 'r+')

    for var_name in nc_file_i.variables:
        file1_var = nc_file_1.variables[var_name]
        filei_var = nc_file_i.variables[var_name]
        #print(var_name,'ndim',average_var.ndim)
        if file1_var.ndim<2 :
            file1_var=file1_var[:12]
            filei_var=filei_var[:12]
            diffs = abs(filei_var-file1_var)
            if np.size((diffs))>1 :
                sumd=np.sum(diffs[:])
            else:
                sumd=diffs
            
                            
            if sumd>0.001:
                print(i, f"Variable {var_name} is different")
                if(np.size((diffs))>1):
                    ln=4
                    print('file_i:',filei_var[1:ln])
                    print('file_1:', file1_var[1:ln])
                    print('diffs:', diffs[1:ln])
                else:
                    print('file_i:',filei_var)
                    print('file_1:', file1_var)
                    print('diffs:', diffs)

 

2 Variable fates_alloc_storage_cushion is different
file_i: [3.09090909 3.09090909 1.54545455]
file_1: [3.13939394 3.13939394 1.56969697]
diffs: [0.04848484848484835 0.04848484848484835 0.024242424242424176]
2 Variable fates_allom_d2bl1 is different
file_i: [0.0809596  0.0809596  0.01156566]
file_1: [0.08378788 0.08378788 0.0119697 ]
diffs: [0.0028282828282828326 0.0028282828282828326 0.00040404040404040664]
2 Variable fates_allom_d2bl2 is different
file_i: [1.87878788 1.87878788 1.87878788]
file_1: [1.34343434 1.34343434 1.34343434]
diffs: [0.5353535353535355 0.5353535353535355 0.5353535353535355]
2 Variable fates_allom_d2ca_coefficient_max is different
file_i: [0.4085476  1.19313622 0.06407376]
file_1: [0.39735451 1.16044756 0.06231831]
diffs: [0.011193084848484858 0.03268866363636347 0.0017554454545454512]
2 Variable fates_allom_fnrt_prof_b is different
file_i: [1.66666667 1.66666667 0.83333333]
file_1: [2.75757576 2.75757576 1.37878788]
diffs: [1.090909090909091 1.090909090909091 0

#### check diffs between SP lhc members

In [29]:

directory_path = '..//lhc_param_files'

file='FATES_LH_347.nc'
file_path = os.path.join(directory_path, file)
data1 = xr.open_dataset(file_path)
file='FATES_LH_348.nc'
file_path = os.path.join(directory_path, file)
data2 = xr.open_dataset(file_path)

common_variables = set(data1.variables) & set(data2.variables)

# Iterate through the common variables and compare their values
parlist=[]
for var_name in common_variables:
    var_data1 = data1[var_name]
    var_data2 = data2[var_name]
    # Compare the values of the variables
    if not var_data1.equals(var_data2):
        parlist.append(var_name)
print(parlist)

['fates_leaf_vcmaxha', 'fates_leaf_stomatal_slope_ballberry', 'fates_leaf_vcmaxse', 'fates_nonhydro_smpsc', 'fates_leaf_stomatal_intercept', 'fates_leaf_jmaxse', 'fates_leaf_vcmaxhd', 'fates_maintresp_leaf_atkin2017_baserate', 'fates_leaf_vcmax25top']


#### Modify SP variables in the LHC ensemble

In [31]:
for i in range(1,nlhc_samples):
    # Make a new filw for each ensemble member
    counter = 0
    # Open the right LHC parameter file
    newfile = pdirroot + str(i) + '.nc' 
    nc_file = nc.Dataset(newfile, 'r+')
    print(newfile)
    #Find an SP parameter set to use 
    
    # Perform operations on each target lhc parameter  
    for index, parameter in enumerate(parlist):
        base_var = nc_file.variables[parameter]
        sp_lhc_var = sp_posterior[i][parameter].values
        
        #print('av',average_var)
        #print('bs',base_var)
        if(base_var.ndim==1):
            base_var[0:12] = sp_lhc_var[0:12]  
        else:
            base_var[:,0:12] = sp_lhc_var[:,0:12]    
      
    nc_file.close()
                

../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_1.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_2.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_3.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_4.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_5.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_6.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_7.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_8.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_9.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_10.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_11.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_12.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_13.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_14.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_15.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_16.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_17.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_18.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_19.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_20.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_21.nc
../paramfiles/NOCOMP_LHC/nocomp_lhc_v1_22.nc
../paramfiles/NOCOM

#### check diffs between lhc members 2nd

In [33]:
newfile = pdirroot + str(1) + '.nc' 
nc_file_1 = nc.Dataset(newfile, 'r+')

for i in range(1,nlhc_samples):
    newfile = pdirroot + str(i) + '.nc' 
    nc_file_i = nc.Dataset(newfile, 'r+')

    for var_name in nc_file_i.variables:
        file1_var = nc_file_1.variables[var_name]
        filei_var = nc_file_i.variables[var_name]
        #print(var_name,'ndim',average_var.ndim)
        if file1_var.ndim<2 :
            file1_var=file1_var[:12]
            filei_var=filei_var[:12]
            diffs = abs(filei_var-file1_var)
            if np.size((diffs))>1 :
                sumd=np.sum(diffs[:])
            else:
                sumd=diffs
            
                            
            if sumd>0.001:
                print(i, f"Variable {var_name} is different")
                if(np.size((diffs))>1):
                    ln=4
                    print('file_i:',filei_var[1:ln])
                    print('file_1:', file1_var[1:ln])
                    print('diffs:', diffs[1:ln])
                else:
                    print('file_i:',filei_var)
                    print('file_1:', file1_var)
                    print('diffs:', diffs)

 

2 Variable fates_alloc_storage_cushion is different
file_i: [3.09090909 3.09090909 1.54545455]
file_1: [3.13939394 3.13939394 1.56969697]
diffs: [0.04848484848484835 0.04848484848484835 0.024242424242424176]
2 Variable fates_allom_d2bl1 is different
file_i: [0.0809596  0.0809596  0.01156566]
file_1: [0.08378788 0.08378788 0.0119697 ]
diffs: [0.0028282828282828326 0.0028282828282828326 0.00040404040404040664]
2 Variable fates_allom_d2bl2 is different
file_i: [1.87878788 1.87878788 1.87878788]
file_1: [1.34343434 1.34343434 1.34343434]
diffs: [0.5353535353535355 0.5353535353535355 0.5353535353535355]
2 Variable fates_allom_d2ca_coefficient_max is different
file_i: [0.4085476  1.19313622 0.06407376]
file_1: [0.39735451 1.16044756 0.06231831]
diffs: [0.011193084848484858 0.03268866363636347 0.0017554454545454512]
2 Variable fates_allom_fnrt_prof_b is different
file_i: [1.66666667 1.66666667 0.83333333]
file_1: [2.75757576 2.75757576 1.37878788]
diffs: [1.090909090909091 1.090909090909091 0